In [1]:
from keras.layers import Dense, Input, Lambda
from keras.layers.wrappers import TimeDistributed
from keras.models import Model
from keras import backend as K
import keras
import tensorflow as tf

Using TensorFlow backend.


In [2]:
keras.__version__, tf.__version__

('2.3.1', '2.0.0')

In [3]:
import numpy as np
from sklearn.datasets import make_classification

In [4]:
X_test, y_test = make_classification(n_samples=1, n_features=40)

X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.int32)

In [5]:
def foo(x):
    B, D = K.int_shape(x)
    if B is None:
        return x
    else:
        mask = tf.random.normal((B,D))
        return x + mask

inputs = Input(shape=(40,))
batch = Lambda(lambda x: K.tile(K.expand_dims(x, 1), [1, 10, 1]))(inputs)
noise = TimeDistributed(Lambda(foo))(batch)
outputs = TimeDistributed(Dense(1))(noise)
model = Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40)                0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 10, 40)            0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 40)            0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 10, 1)             41        
Total params: 41
Trainable params: 41
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(optimizer='adam', loss='binary_crossentropy')

In [7]:
K.eval(model(tf.convert_to_tensor(X_test)))

array([[[ 1.0880609 ],
        [-0.03239107],
        [ 3.2225313 ],
        [ 3.4127264 ],
        [-0.40568006],
        [ 2.3533857 ],
        [ 4.065094  ],
        [-0.07377666],
        [ 0.09491897],
        [ 1.1561517 ]]], dtype=float32)

In [8]:
model.predict(X_test)

array([[[1.4695077],
        [1.4695077],
        [1.4695077],
        [1.4695077],
        [1.4695077],
        [1.4695077],
        [1.4695077],
        [1.4695077],
        [1.4695077],
        [1.4695077]]], dtype=float32)

In [9]:
model.save_weights('random_noise.keras')

# The same model without noise

In [10]:
inputs = Input(shape=(40,))
batch = Lambda(lambda x: K.tile(K.expand_dims(x, 1), [1, 10, 1]))(inputs)
outputs = TimeDistributed(Dense(1))(batch)
model = Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 40)                0         
_________________________________________________________________
lambda_3 (Lambda)            (None, 10, 40)            0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 10, 1)             41        
Total params: 41
Trainable params: 41
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.load_weights('random_noise.keras')

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy')

In [13]:
K.eval(model(tf.convert_to_tensor(X_test)))

array([[[1.4695077],
        [1.4695077],
        [1.4695077],
        [1.4695077],
        [1.4695077],
        [1.4695077],
        [1.4695077],
        [1.4695077],
        [1.4695077],
        [1.4695077]]], dtype=float32)

In [14]:
model.predict(X_test)

array([[[1.4695077],
        [1.4695077],
        [1.4695077],
        [1.4695077],
        [1.4695077],
        [1.4695077],
        [1.4695077],
        [1.4695077],
        [1.4695077],
        [1.4695077]]], dtype=float32)